In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

In [2]:
path = '../data/lemon_haze_18.csv'
test_path = '../data/test_shard.csv'

In [355]:
from convert_to_time_series import TimeSeriesData

In [356]:
lemon_haze = TimeSeriesData(path)

In [357]:
%time lemon_haze.construct_tables()

CPU times: user 135 ms, sys: 75.7 ms, total: 211 ms
Wall time: 276 ms


In [358]:
raw = lemon_haze.raw_df
inv = lemon_haze.by_inv_id
ts = lemon_haze.time_series
bl = lemon_haze._bool_matrix
sls = lemon_haze._sales_matrix
dts = lemon_haze._date_idx

In [295]:
bl.shape, sls.shape, ts.shape

((1112, 2514), (2514, 2), (1112, 2))

In [359]:
ts.head()

,ttl_sales,ttl_gross_profit
2014-07-17,120.945312,78.757812
2014-07-18,120.945312,78.757812
2014-07-19,354.418997,249.073602
2014-07-20,354.418997,249.073602
2014-07-21,354.418997,249.073602


In [281]:
inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2514 entries, 0 to 2513
Data columns (total 11 columns):
wa_inventory_id      2514 non-null int64
generic_strain_id    2514 non-null int64
first_rtl_sale       2514 non-null datetime64[ns]
latest_rtl_sale      2514 non-null datetime64[ns]
days_sales           2514 non-null timedelta64[ns]
wholesale_cogs       2514 non-null float64
ttl_rtl_sales        2514 non-null float64
gross_profit         2514 non-null float64
avg_daily_sales      2513 non-null float64
avg_daily_gross      2513 non-null float64
units_sold           2514 non-null int64
dtypes: datetime64[ns](2), float64(5), int64(3), timedelta64[ns](1)
memory usage: 216.1 KB


In [283]:
bl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Columns: 2514 entries, 0 to 2513
dtypes: bool(2514)
memory usage: 2.7 MB


In [284]:
sls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2514 entries, 0 to 2513
Data columns (total 2 columns):
0    2513 non-null float64
1    2513 non-null float64
dtypes: float64(2)
memory usage: 39.4 KB


In [285]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1112 entries, 2014-07-17 to 2017-08-01
Freq: D
Data columns (total 2 columns):
ttl_sales           0 non-null float64
ttl_gross_profit    0 non-null float64
dtypes: float64(2)
memory usage: 26.1 KB


In [352]:
x = '1 days'

In [353]:
inv.loc[inv['days_sales'].apply(lambda x: x.days) == 0, 'days_sales'] = \
    pd.to_timedelta(x)

In [354]:
inv['days_sales']

0      64 days
1      19 days
2       2 days
3       2 days
4       1 days
5       5 days
6       1 days
7       8 days
8       8 days
9      11 days
10      7 days
11      8 days
12      6 days
13     11 days
14     13 days
15     14 days
16     11 days
17      9 days
18     20 days
19      5 days
20      3 days
21      1 days
22     16 days
23     12 days
24     24 days
25      1 days
26     30 days
27     27 days
28      8 days
29      6 days
         ...  
2484    1 days
2485    1 days
2486    1 days
2487    1 days
2488    1 days
2489    4 days
2490    4 days
2491    4 days
2492    4 days
2493    2 days
2494    1 days
2495    1 days
2496    3 days
2497    1 days
2498    1 days
2499    3 days
2500    2 days
2501    2 days
2502    1 days
2503    1 days
2504    2 days
2505    1 days
2506    2 days
2507    3 days
2508    1 days
2509    1 days
2510    1 days
2511    1 days
2512    1 days
2513    1 days
Name: days_sales, Length: 2514, dtype: timedelta64[ns]